# 최적화 분할

지역적 명암 변화만 살폈던 분할 알고리즘에서는 양말의 색이 배경색과 비슷하면 양말 부분의 경계가 형성되지 않아 사람 영역이 배경과 섞입니다.

이 문제를 해결하는 방법은 지역적 명암 변화를 보되 전역적 정보를 함께 고려하는 것입니다.

비록 지역적으로 아주 약한 색상 변화지만 전역적으로 유리한 측면이 있다면 그곳을 경계로 간주합니다.

이 발상의 구현은 1.영상을 그래프로 표현 2.분할을 최적화 문제로 해결하는 것입니다.

### 영상의 그래프 표현

그래프 이론에서 그래프는 노드(vertex)와 에지(edge)로 구성됩니다.

<img src="graph.png" alt="Graph" width="500"/>

영상을 그래프로 표현할 때는 화소나 슈퍼 화소를 노드로 취합니다.

400x600 크기의 이미지에서 화소를 노드로 사용하면 노드가 24만 개이므로 계산 효율에서 좋지 못합니다.

> 따라서 일반적으로 슈퍼 화소를 노드로 취합니다.

이미지에서 이웃한 두 노드 간의 연결을 에지로 나타낼 수 있습니다.

이때 에지의 가중치는 두 노드 간의 유사성을 나타내는 값입니다.(예: 색상 차이, 거리)

> 여기에서는 유사도를 가중치로 사용합니다.

### 두 노드 간의 거리
두 노드 $v_{p}$와 $v_{q}$ 사이의 거리 $d_{pq}$는 아래의 식과 같이 계산됩니다.

> $d_{pq} = \left\{\begin{matrix} \left\|f(v_{p})-f(v_{q})\right\|, & \text{if }v_{q}\in\text{neighbor}(v_{p})\\ \infty, & \text{else} \end{matrix}\right.$

$v_{q}\in\text{neighbor}(v_{p})$는 $v_{p}$와 $v_{q}$가 8-이웃을 이루거나 둘 사이의 거리가 사용자가 지정한 값 $r$이내면 참입니다.

$v_{q}\in\text{neighbor}(v_{p})$를 만족하면 두 노드 사이의 거리는 두 노드의 벡터 $f(v_{p})$와 $f(v_{q})$간의 거리입니다.

$v_{q}\in\text{neighbor}(v_{p})$를 만족하지 못하면 두 노드 사이의 거리는 $\infty$ 입니다.

$\left\|f(v_{p})-f(v_{q})\right\|$은 벡터 간의 거리를 계산하는 수식입니다.

이는 각 벡터 간의 성분 차이를 계산하고, 차이 벡터의 제곱합의 제곱근을 구하는 과정을 통해 거리를 계산합니다.

$f(v)$는 $v$에 해당하는 화소의 색상과 위치를 결합한 벡터입니다.

예를들어 $v$가 $(10,12)$에 위치한 화소이고 RGB 색상이 $(13,60,55)$인 경우 $f(v) = (13,60,55,10,12)$ 입니다.

슈퍼 화소를 노드로 사용하는 경우에는 슈퍼 화소를 구성하는 화소들의 평균 벡터를 사용합니다.

### 두 노드 간의 유사도
유사도는 거리와 반대되는 개념이므로 거리 $d_{pq}$가 가질 수 있는 최대값 $D$에서 $d_{pq}$를 뺀 값 또는 $d_{pq}$의 역수를 사용합니다.

$v_{q}\in\text{neighbor}(v_{p})$를 만족하지 못하면 두 노드 간의 유사도는 $0$ 입니다.

두 노드 $v_{p}$와 $v_{q}$ 사이의 유사도 $s_{pq}$는 아래의 식과 같이 계산됩니다.

> $s_{pq} = \left\{\begin{matrix} D-d_{pq} \text{ or } \frac{1}{e^{d_{pq}}}, & \text{if }v_{q}\in\text{neighbor}(v_{p})\\ 0, & \text{else} \end{matrix}\right.$

### 정규화 절단 알고리즘
2000년에 발표된 정규화 절단(normalized cut)은 화소를 노드로 취하고, $f(v)$로 색상과 위치를 결합한 5차원 벡터를 사용하며, 유사도는 에지 가중치로 사용한다.

커트(cut)는 그래프를 두 개의 부분으로 나누는 에지들의 집합인데, 원래 영역을 2개의 영역 $C_{1}$과 $C_{2}$로 분할했을 때 집합 $cut$의 수식은 아래와 같다.

> $cut(C_{1},C_{2}) = \sum_{v_{p}\in C_{1},v_{q}\in C_{2}}S_{pq}$

두 영역 $C_{1}$과 $C_{2}$이 제대로 된 분할이라면 $C_{1}$에 속한 $v_{p}$와 $C_{2}$에 속한 $v_{q}$의 유사도가 작아 $cut(C_{1},C_{2})$는 작다.

반대로 좋지 않은 분할의 $cut(C_{1},C_{2})$는 크다. 다시 말해 $cut$은 영역 분할의 좋고 나쁨의 정도를 측정해주는 목적 함수이다.

$cut$은 $C_{1}$과 $C_{2}$ 영역이 클수록 둘 사이에 에지가 많으므로 덩달아 커진다. (에지 가중치의 합이므로)

결국 $cut$을 목적 함수로 사용한 알고리즘은 영역을 자잘하게 분할하는 경향을 띈다.

아래의 식은 $cut$을 정규화 한것이다. $C$는 $C_{1}$과 $C_{2}$로 나뉘기 전의 원래 영역이다.

> $ncut(C_{1},C_{2}) = \frac{cut(C_{1},C_{2})}{cut(C_{1},C)} + \frac{cut(C_{1},C_{2})}{cut(C_{2},C)}$

정규화 절단은 그래프를 자잘하게 분할하지 않고 균형 있게 분할하기 위해 각 영역의 내부 연결성을 최대화하고 외부 연결성을 최소화하는 것을 목표로 한다.

$ncut$을 목적 함수로 사용하면 분할을 최적화 문제로 풀 수 있고, $ncut$이 작을수록 좋은 분할이므로 최소화 문제이다.

원래 영역 $C$에서 $ncut$이 최소인 분할 $C_{1}$과 $C_{2}$를 어떻게 찾을 것인가?

정규화 절단 알고리즘이 발표되고 10여 년이 지나 Shi는 $ncut$이 최소에 가깝도록 분할하는 효율적인 방법을 제안한다.

Shi와 같은 연구실의 Arbelaez 등이 개발한 해당 알고리즘은 고전적 영역 분할 알고리즘 중에 가장 뛰어난 것으로 평가된다. 이는 추후에 다룬다.